 # Welcome to the Hartzell Python Stock Portfolio Tracker!

   The design is simple. It is comprised on one main interface that breaks off into four: the buying interface, selling interface, the display interface, and edit interface. You can record your buys and sells within the respective interface and the code will automatically fill in an excel sheet with things such as average fulfillment price, profit/loss, and capital. The display interface is the most complex interface. It is comprised on four options: display sheet, display graph, diversification information, and advanced statsitcs. The display sheet method simply opens another interface and displays the sheet in a readable format. Display graph will open a new interface of a capital v. time graph as well as provide information about the graph. Diversification information displays valuable information about where profits are coming from. Lastly, advanced statistics shows finance-related numbers. The last interface, edit, simply allows the user to display and edit their sheet by clearing it, deleting a row, or changing a cell specifically.The interface also will allow the user to edit the excel sheet manually and still run. After the user is done using the interface, their sheet is automatically exported onto their devices as "finance.xlsx".

This project took long hours and incorporates various python libraries such as pandas, tkinter, openpyxl, matplotlib, and datetime. 

 *** Please run the "prereq" file before this to ensure that the template is downloaded. ***

Notes:

I put in 2 months worth of my personal trading as an example, so you can see how it functions! 

# Importing Libraries

Libraries Used: pandas, tkinter, matplotlib, matplotlib.figure, openpyxl, datetime

In [1]:
# Importing Libraries
import pandas as pd
import tkinter as tk
import matplotlib.pyplot as plt
import datetime
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import Font
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import statistics
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from tkinter import ttk
from PIL import Image, ImageTk

# Loading Excel File

 Loading our workbook into the excel file allows for you to go back after using to GUI or make edits directly on Excel. This works with the saving feature to have the program function. 

In [2]:
# Loading our workbook into the excel file
# Allows for you to go back after using to GUI or make edits directly on Excel
finance = load_workbook('Finance.xlsx')
sheet = finance.active

# Methods

In order to make the program shorter and easier to understand, I made multiple methods for use. Some are extremely lengthy and specified, while others are shorter but are used extremely often.

In [3]:
# Advanced Statistics: 
# Calculate and return total profits, profits per day, profits per trade, and standard deviation of excess return
def advanced_statistics():
    
    # Initalize variable
    total_profits = 0
    trades = 0
    capital = []
    
    # Iterates through all trades
    for row in range(3,sheet.max_row+1):
        
        # For each trade, add one to trade, add profits/losses to the total, and adds the individual profts or losses to a list
        total_profits += sheet['J' + str(row)].value
        trades += 1
        capital.append(sheet['K' + str(row)].value/1000)
    
    # Calculates how many days has gone by from the beggining to the end, inclduing weekends
    startdate = sheet['B3'].value
    enddate = sheet['B' + str(sheet.max_row)].value
    business_days = CustomBusinessDay(calendar = USFederalHolidayCalendar())
    diff = len(pd.date_range(start = startdate, end=enddate, freq = business_days))
    
    # Calculates profits per trade, rounds to nearest cent
    prof_per_trade = total_profits / trades
    prof_per_trade = round(prof_per_trade,2)
    
    # Calculates profits per day, rounds to nearest cent
    prof_per_day = total_profits / diff
    prof_per_day = round(prof_per_day,2)
    
    # Calculates standard deviation of profits (on each trade)
    dev = statistics.stdev(capital)
    dev = round(dev,2)
    
    # Round total profits to nearest cent
    total_profits = round(total_profits,2)
    #Returns variables
    return total_profits,prof_per_trade,prof_per_day,dev

In [4]:
# Calculates and returns variables for each stock, used in diversification information option
def get_divers():
    
    # Initalizes Dict: Dictionary of each stock(key) and its profits or losses(value)
    dict = {}
    
    # Initalizes Dict2: Dictionary of each stock(key) and the number of trades made on it (value)
    dict2 = {}
    
    # Goes through each trade and adds the right values to the right dictionaries
    for row in range(3,sheet.max_row+1):
        ticker = str(sheet['A' + str(row)].value)
        if ticker not in dict.keys():
            dict[ticker] = float(sheet['J' + str(row)].value)
            dict2[ticker] = 1
        else:
            dict[ticker] += float(sheet['J' + str(row)].value)
            dict2[ticker] += 1
    
    # Returns dictionary
    return dict,dict2

In [5]:
# Calculates Rate of Return on Investment from start to end
def ROR():
    start = float(sheet['K2'].value)
    max_row = int(sheet.max_row)
    end = float(sheet['K' + str(max_row)].value)
    return end/start -1 

In [6]:
# Convert String date to dateime date: Useful in various methods
def format_date(unformat):
    format = datetime.datetime.strptime(unformat, '%b %d %y')
    return format

In [7]:
# Returns total amount of capital after all trades have been made
def get_capital():
    start = float(sheet['K2'].value)
    for row in range(3,sheet.max_row+1):
        start += float(sheet['J' + str(row)].value)
    return start


In [8]:
# Updates capital column for all cells
def update_capital():
    for row in range(3,sheet.max_row+1):
        sum = float(sheet['K2'].value)
        for num in range(3,row+1):
            sum += float(sheet['J' + str(num)].value)
        sheet['K' + str(row)] = sum

In [9]:
# Used to approximate value of portfolio one year from now
def extrapolate():
    #Creates start and end dates and values on those dates
    start = float(sheet['K2'].value)
    startdate = sheet['B3'].value
    maxr = sheet.max_row
    end = float(sheet['K' + str(maxr)].value)
    enddate = sheet['B' + str(maxr)].value
    
    # Finds difference between start and end capital 
    diff = end-start
    diff2= enddate-startdate
    
    # Calculates, returns extrpolated value
    val = 365.0/ int(diff2.days)
    val = val * diff
    return val

In [10]:
# Buy Method: Called in BUY GUI
def buy(date,ticker,shares,price):
    
    added = False
    print('Processing purchase of ' + str(shares) + ' of ' + ticker + ' at ' + str(price)) 
    
    # Iterates through each row of sheet
    for row in range(3,sheet.max_row+1):
        tick = str(sheet['A' + str(row)].value)
        qrem = int(sheet['H' + str(row)].value)
        
        # If the ticker already has an open order, we add the values of this buy to that one
        if(tick == ticker and qrem > 0):
            print(ticker + ' has been found as an open order in row ' + str(row) + '. Adding to that order.')
            qbought = int(sheet['C' + str(row)].value)
            pbought = float(sheet['D' + str(row)].value)
            old_cap = qbought * pbought
            new_cap = shares * price
            sheet['C'+str(row)] = shares + qbought
            sheet['H' + str(row)] = shares + qbought
            sheet['D' + str(row)] = (old_cap+new_cap)/(sheet['C'+str(row)].value)
            sheet['B' + str(row)] = date
            added = True
    
    # If the ticker is not found as an open order, we make a new row
    if added == False:
        print(ticker + ' Not Found in Open Orders ... Adding New Order!')
        sheet.append([ticker,date,shares,price,'N/A',0,0,shares,0,0,get_capital()])

In [11]:
# Sell method: used in sell option
def sell(date,ticker,shares,price):
    added = False
    print('Processing ' + str(shares) +' share sell of ' + str(ticker) + ' at ' + str(price))
    
    # Iterates through each row of sheet
    for row in range(3,sheet.max_row+1):
        tick = str(sheet['A' + str(row)].value)
        qrem = int(sheet['H' + str(row)].value)
        qbought = int(sheet['C' + str(row)].value)
        qsold = int(sheet['F' + str(row)].value)
        
        # Find the ticker in open orders
        if tick == ticker and qrem>0:
            added= True
            print('Found in row ' + str(row) + ' of excel sheet!')
            sheet['E' + str(row)] = date
            sheet['F' + str(row)] = qsold + shares
            sheet['H' + str(row)] = qbought - shares - qsold
            
            # If the Ticker is found as a stock that has been partially sold already, do special calculations
            if qsold>0:
                old_cap = qsold * float(sheet['G' + str(row)].value)
                new_cap = shares * price
                qsold = sheet['F' + str(row)].value
                sheet['G' + str(row)] = (old_cap + new_cap) / qsold
                net = float(sheet['G' + str(row)].value) - float(sheet['D' + str(row)].value)
                sheet['I' + str(row)] = net / float(sheet['D' + str(row)].value)
                sheet['J' + str(row)] = net * qsold
                sheet['K' + str(row)] = get_capital()
                
            # Else, do standard calculations
            else:
                sheet['G' + str(row)] = price
                net = float(sheet['G' + str(row)].value) - float(sheet['D' + str(row)].value)
                
                # Special case for dividends / free stock
                if(sheet['D' + str(row)].value == 0):
                    sheet['I' + str(row)] = 0
                    sheet['J' + str(row)] = float(sheet['G' + str(row)].value)
                else:
                    sheet['I' + str(row)] = net
                    sheet['J' + str(row)] = net * int(sheet['F'+str(row)].value)
                sheet['K' + str(row)] = get_capital()
    
    # If the ticker was never found, print that it wasn't found
    if added == False:
        print(ticker + ' was not found in any open orders. Please try again.')
        
    # Update capital for all rows
    update_capital()
                

# User Interfaces

I used multiple different interfaces that branch off into more. The Interfaces are listed in the most logical order, with main branching off into its 4 options and display doing the same

In [98]:
class Main_GUI:
    def __init__(self,master):
        
        # Sets background color
        master['bg'] = 'black'
        
        # Initalizes First Label: Intro
        self.label1 = tk.Label(master, 
                               text = 'Hello Gavin, What would you like to do today?', 
                               bg = 'black', 
                               fg = '#D4AF37',
                               font = 'Arial 13 bold italic',
                               )
        
        # Pack Label into GUI
        self.label1.pack(expand= True,
                        pady = 8)
        
        # Finads logo image
        i = Image.open('logo2.png')
        size = (156,156)
        i.thumbnail(size)
        img = ImageTk.PhotoImage(i)
        
        # Creates and packs label for image
        panel = tk.Label(master,
                         image = img,
                         width = 156,
                         height=155,
                         borderwidth = 0)
        panel.image = img
        panel.pack(side = tk.TOP, expand=True)
        
        # Creates Button 1: Record Buy
        self.button1 = tk.Button(master,
                                 text= 'Record Buy', 
                                 command = self.record_buy, 
                                 bg = 'black', 
                                 fg = '#D4AF37', 
                                 font = 'Arial 10 italic',
                                 highlightcolor = '#D4AF37',
                                 highlightbackground = '#D4AF37',
                                 highlightthickness = 1)
        
        # Pack Button 1 Into GUI under Label
        self.button1.pack(expand = True,
                         pady=6)
        
        # Creates Button 2: Record Sell
        self.button2 = tk.Button(master,
                                 text = 'Record Sell',
                                 command = self.record_sell,
                                 bg = 'black', 
                                 fg = '#D4AF37',
                                 font = 'Arial 10 italic',
                                 highlightcolor = '#D4AF37',
                                 highlightbackground = '#D4AF37',
                                 highlightthickness = 1)
        
        # Pack Button 2 Into GUI under Buy Button
        self.button2.pack(expand = True,
                         pady=6)
        
        # Creates Button 3: Display Information
        self.button3 = tk.Button(master,
                                 text= 'Display Information', 
                                 command = self.display_info,
                                 bg = 'black', 
                                 fg = '#D4AF37',
                                 font = 'Arial 10 italic',
                                 highlightcolor = '#D4AF37',
                                 highlightbackground = '#D4AF37',
                                 highlightthickness = 1)
        # Packs Button 3 into GUI under Sell Button
        self.button3.pack(expand = True, 
                          pady=6)
        
        # Creates Button 4: Edit Excel Sheet
        self.button4 = tk.Button(master,
                                 text= 'Edit Excel Sheet', 
                                 command = self.edit_sheet, 
                                 bg = 'black', 
                                 fg = '#D4AF37',
                                 font = 'Arial 10 italic',
                                 highlightcolor = '#D4AF37',
                                 highlightbackground = '#D4AF37',
                                 highlightthickness = 1)
        
        # Packs Sheet into GUI under Sell Button
        self.button4.pack(expand = True,
                         pady=6)
        
    # Buy Method
    def record_buy(self):
        
        # Opens Buy_GUI
        buy_gui = Buy_GUI()
        buy_gui.mainloop()
        
    # Sell Method
    def record_sell(self):
        
        # Opens Sell_GUI
        sell_gui = Sell_GUI()
        sell_gui.mainloop()
        
    # Display Method
    def display_info(self):
        
        # Opens Display_GUI
        dis_gui = Display_GUI()
        dis_gui.mainloop()
        
    # Edit Method
    def edit_sheet(self):
        
        # Opens Edit_GUI
        ed_gui = Edit_GUI()
        ed_gui.mainloop()

In [156]:
class Buy_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets background color
        self.title('Buy Interface')
        self['bg'] = 'black'
        
        # Creates 3 Frames: Intro, Options, and Button
        self.intro_frame = tk.Frame(self, background = 'black')
        self.options_frame = tk.Frame(self, background = 'black')
        self.button_frame = tk.Frame(self, background = 'black')
        
        # Creates Label for intro, packs into intro frame
        label_for_title = tk.Label(self.intro_frame,
                                   text = 'Please Enter the Following:',
                                   bg = 'black',
                                   fg = '#e8b923',
                                   highlightcolor = 'black',
                                   font = 'Arial 12 bold italic',
                                   highlightbackground = 'black',
                                   highlightthickness = 1)
        label_for_title.pack(pady =12,expand=True)
        
        # Packs intro frame
        self.intro_frame.pack(expand=True)
        
        # Create Date Label and Date Entry, packs into options frame
        self.datelabel = tk.Label(self.options_frame, 
                                  text = 'Date',
                                  bg = 'black', 
                                  fg = '#e8b923', 
                                  font = 'Arial 11  bold italic')
        self.datelabel.grid(row = 0, column = 0)
        self.enterdate = tk.Entry(self.options_frame, 
                                  textvariable = 'Date',
                                  bg = '#e8b923',
                                  borderwidth =0)
                                
        self.enterdate.grid(row=0, column = 1,pady = 8,padx = 5)
        
        # Create Ticker Label and Ticker Entry, packs into options frame
        self.tickerlabel = tk.Label(self.options_frame, 
                                    text = 'Ticker',
                                    bg = 'black', 
                                    fg = '#e8b923', 
                                    font = 'Arial 11  bold italic')
        self.tickerlabel.grid(row = 1, column = 0)
        self.enterticker = tk.Entry(self.options_frame, 
                                    textvariable = 'Ticker',
                                    bg = '#e8b923',
                                    borderwidth = 0)
        self.enterticker.grid(row=1, column = 1,pady = 8,padx = 5)
        
        # Create Shares Label and Shares Entry, packs into options frame
        self.sharelabel = tk.Label(self.options_frame, 
                                   text = 'Shares',
                                   bg = 'black', 
                                   fg = '#e8b923', 
                                   font = 'Arial 11  bold italic')
        self.sharelabel.grid(row = 2, column = 0)
        self.entershares = tk.Entry(self.options_frame,
                                    textvariable = 'Shares',
                                    bg = '#e8b923',
                                    borderwidth = 0)
        self.entershares.grid(row=2, column = 1,pady = 8,padx = 5)
        
        # Create Price Label and Price Entry, packs into options frame
        self.pricelabel = tk.Label(self.options_frame, 
                                   text = 'Price',
                                   bg = 'black', 
                                   fg = '#e8b923', 
                                   font = 'Arial 11  bold italic')
        self.pricelabel.grid(row = 3, column = 0)
        self.enterprice = tk.Entry(self.options_frame,
                                   textvariable = 'Price',
                                   bg = '#e8b923',
                                   borderwidth = 0)
        self.enterprice.grid(row=3, column = 1,pady = 8,padx = 5)
        
        # Packs options frame
        self.options_frame.pack(expand=True)
        
        # Create 'Record Buy' button, packs into button frame, calls buy_method
        self.buy_button = tk.Button(self.button_frame,
                                    text = 'Record Buy', 
                                    command = self.buy_method,
                                    bg = 'black', 
                                    fg = '#e8b923', 
                                    font = 'Arial 10 italic',
                                    highlightcolor = '#e8b923',
                                    highlightbackground = '#e8b923',
                                    highlightthickness = 1)
        self.buy_button.pack(expand=True,pady = 8)
        
        # Packs button frame
        self.button_frame.pack(expand=True)
        
        # Creates confirmation frame
        self.conf = tk.Frame(self, background = 'black')
        self.conf.pack()
        
    # Method to record buy
    def buy_method(self):
        
        # Converts entry variables into actual variables
        unformatted_date = str(self.enterdate.get())
        date = format_date(unformatted_date)
        ticker = str(self.enterticker.get())
        shares = int(self.entershares.get())
        price = float(self.enterprice.get())
        
        # Calls buy method with transformed variables
        buy(date,ticker,shares,price)
        
        # Confirms entry, packs confirmation
        conf_label = tk.Label(self.conf,
                              text = 'Purchase of ' + str(shares) + " shares of " + ticker + ' at ' + str(price) + ' has been recorded!',
                              bg = 'black',
                              fg = '#e8b923',
                              font = 'Arial 10 italic')
        print()
        conf_label.pack(expand=True)

In [157]:
class Sell_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets background color
        self['bg'] = 'black'
        
        # Creates 3 Frames: Intro, Options, and Button
        self.intro_frame = tk.Frame(self,background = 'black')
        self.options_frame = tk.Frame(self,background = 'black')
        self.button_frame = tk.Frame(self,background = 'black')
        
        # Creates Label for intro, packs into intro frame
        label_for_title = tk.Label(self.intro_frame, 
                                   text = 'Please Enter the Following:',
                                   bg = 'black', 
                                   fg = '#e8b923', 
                                   font = 'Arial 11  bold italic')
        label_for_title.pack(pady =12,expand=True)
        
        # Packs intro frame
        self.intro_frame.pack(expand=True)
        
        # Create Date Label and Date Entry, packs into options frame
        self.datelabel = tk.Label(self.options_frame, 
                                  text = 'Date',
                                  bg = 'black', 
                                  fg = '#e8b923', 
                                  font = 'Arial 11  bold italic')
        self.datelabel.grid(row = 0, column = 0)
        self.enterdate = tk.Entry(self.options_frame,
                                   textvariable = 'Date',
                                   bg = '#e8b923',
                                   borderwidth =0)
        self.enterdate.grid(row=0, column = 1,pady = 8,padx = 5)
        
        # Create Ticker Label and Ticker Entry, packs into options frame
        self.tickerlabel = tk.Label(self.options_frame, 
                                    text = 'Ticker',
                                    bg = 'black', 
                                    fg = '#e8b923', 
                                    font = 'Arial 11  bold italic')
        self.tickerlabel.grid(row = 1, column = 0)
        self.enterticker = tk.Entry(self.options_frame, 
                                    textvariable = 'Ticker',
                                    bg = '#e8b923',
                                    borderwidth =0)
        self.enterticker.grid(row=1, column = 1,pady = 8,padx = 5)
        
        # Create Shares Label and Shares Entry, packs into options frame
        self.sharelabel = tk.Label(self.options_frame, 
                                   text = 'Shares',
                                   bg = 'black', 
                                   fg = '#e8b923', 
                                   font = 'Arial 11  bold italic')
        self.sharelabel.grid(row = 2, column = 0)
        self.entershares = tk.Entry(self.options_frame,
                                     textvariable = 'Shares',
                                     bg = '#e8b923',
                                     borderwidth =0)
        self.entershares.grid(row=2, column = 1,pady = 8,padx = 5)
        
        # Create Price Label and Price Entry, packs into options frame
        self.pricelabel = tk.Label(self.options_frame, 
                                   text = 'Price',
                                   bg = 'black', 
                                   fg = '#e8b923', 
                                   font = 'Arial 11  bold italic')
        self.pricelabel.grid(row = 3, column = 0)
        self.enterprice = tk.Entry(self.options_frame,
                                   textvariable = 'Price',
                                   bg = '#e8b923',
                                   borderwidth =0
                                  )
        self.enterprice.grid(row=3, column = 1,pady = 8,padx = 5)
        
        # Packs options frame
        self.options_frame.pack(expand=True)
        
        # Creates and packs sell button into button frame, calls sell_method
        self.sell_button = tk.Button(self.button_frame, 
                                      text = 'Record Sell', 
                                      command = self.sell_method,
                                      bg = 'black', 
                                      fg = '#e8b923', 
                                      font = 'Arial 10 italic',
                                      highlightcolor = '#e8b923',
                                      highlightbackground = '#e8b923',
                                      highlightthickness = 1)
        self.sell_button.pack(pady=8,expand=True)
        
        # Packs button frame
        self.button_frame.pack(pady=8,expand=True)
        
        # Creates frame for confirmation label
        self.conf = tk.Frame(self, background = 'black')
        self.conf.pack()

    # Records Sell
    def sell_method(self):
        
        # Converts entry variables into useful variables
        unformatted_date = str(self.enterdate.get())
        date = format_date(unformatted_date)
        ticker = str(self.enterticker.get())
        shares = int(self.entershares.get())
        price = float(self.enterprice.get())
        
        # Records sell with tranformed variables
        sell(date,ticker,shares,price)
        
        # Prints confirmation label, packs into confirmation frame
        conf_label = tk.Label(self.conf,
                              text = 'Sold ' + str(shares) + " shares of " + ticker + ' at ' + str(price) + '!',
                              bg = 'black',
                              fg = '#e8b923',
                              font = 'Arial 10 italic')
        print()
        conf_label.pack()

In [129]:
class Display_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets background color
        self['bg'] = 'black'
        
        # Creates intro label and packs into frame
        label = tk.Label(self,
                        text = 'Hello Gavin, welcome to the display page!',
                        font = 'Arial 12 bold italic',
                        bg = 'black',
                        fg = '#e8b923')
        label.pack(expand=True,pady=12)
        
        # Creates display sheet button, packs into self, calls excel
        display = tk.Button(self,
                            text = 'Display Excel Sheet',
                            command = self.excel,
                            bg = 'black', 
                            fg = '#e8b923', 
                            font = 'Arial 10 italic',
                            highlightcolor = '#e8b923',
                            highlightbackground = '#e8b923',
                            highlightthickness = 1)
        display.pack(expand=True,pady=8)
        
        # Creates display graph button, packs into self, calls graph
        graph = tk.Button(self, 
                          text = 'Display Graph', 
                          command = self.graph,
                          bg = 'black', 
                          fg = '#e8b923', 
                          font = 'Arial 10 italic',
                          highlightcolor = '#e8b923',
                          highlightbackground = '#e8b923',
                          highlightthickness = 1)
        graph.pack(expand=True,pady=8)
        
        # Creates Diversification Information button, packs into self, calls diverse
        div = tk.Button(self,
                        text = 'Stock Diversification',
                        command = self.diverse,
                        bg = 'black', 
                        fg = '#e8b923', 
                        font = 'Arial 10 italic',
                        highlightcolor = '#e8b923',
                        highlightbackground = '#e8b923',
                        highlightthickness = 1)
        div.pack(expand=True,pady=8)
        
        # Creates "Advanced Statistics" button, packs into self, calls sharpe
        sharpe = tk.Button(self,
                           text = 'Advanced Statistics', 
                           command = self.sharpe,
                           bg = 'black', 
                           fg = '#e8b923', 
                           font = 'Arial 10 italic',
                           highlightcolor = '#e8b923',
                           highlightbackground = '#e8b923',
                           highlightthickness = 1)
        sharpe.pack(expand=True,pady=8)
    
    # Creates Display Sheet GUI
    def excel(self):
        ex_gui = Excel_GUI()
        ex_gui.mainloop()
    
    # Creates Display Graph GUI
    def graph(self):
        graph_gui = Graph_GUI()
        graph_gui.mainloop()
        
    # Creates Diversification Information GUI
    def diverse(self):
        diverse_gui = div_GUI()
        diverse_gui.mainloop()
        
    # Creates Advanced Statistics GUI
    def sharpe(self):
        stats_gui = stats_GUI()
        stats_gui.mainloop()

In [130]:
class Graph_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Set GUI Background
        self['bg'] = 'black'
        
        # Finds, labels, packs approximate value of porfolio one year from now
        lab2 = tk.Label(self, 
                        text = 'Approximate Value 1 Year from Now: $' + str(round(extrapolate(),2)),
                        bg = 'black',
                        fg = '#e8b923',
                        font = 'Arial 12 bold italic')
        lab2.pack(expand=True)
        
        # Finds, labels, packs approximate Rate of Return
        lab3 = tk.Label(self, 
                        text = 'Average ROR: ' + str(round(ROR(),2)) + '%',
                        bg = 'black',
                        fg = '#e8b923',
                        font = 'Arial 12 bold italic')
        lab3.pack(expand=True)
        
        # Prints graph title
        lab = tk.Label(self, 
                       text = 'Capital v. Time',
                       bg = 'black',
                       fg = '#e8b923',
                       font = 'Arial 12 bold italic')
        lab.pack(expand=True)
        
        # FIlls 2 lists, x and y, with time variables and capital vairbales, respectively
        x=[]
        y=[]
        for row in range(3, sheet.max_row+1):
            x.append(sheet['B' + str(row)].value)
            y.append(sheet['K' + str(row)].value)
            
        # Creates matplotlib figure and subplot
        f = Figure(figsize = (8,8),dpi = 100)
        f.patch.set_facecolor('white')
        a = f.add_subplot(111)
        
        a.plot(x,y)
        a.set_xlabel('Date')
        a.set_ylabel('Capital')
        
        # Creates canvas for figure, prints figure on canvas, packs canvas
        canvas = FigureCanvasTkAgg(f,self)
        canvas.draw()
        canvas.get_tk_widget().pack(side = tk.BOTTOM,fill = tk.BOTH, expand = True)


In [161]:
# Diversification Information GUI:
class div_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets GUI Background
        self['bg'] = 'black'
        
        # Creates and packs Title Label
        titl = tk.Label(self, 
                        text = 'Profits/ Losses Per Stock',
                        bg = 'black',
                        fg = '#e8b923',
                        font = 'Arial 12 bold italic')
        titl.pack(expand=True,pady=4)
        
        # Uses get_divers() method above
        dict,dict2 = get_divers()
        
        # Iterates through dictionary 1 to print each stocks profit or loss
        for key in dict.keys():
            lab = tk.Label(self,
                           text = key + ': ' + str(round(dict[key],2)),
                           bg = 'black',
                           fg = '#e8b923',
                           font = 'Arial 10 bold italic')
            lab.pack(expand=True)
        
        # Creates and packs second title label
        titl2 = tk.Label(self, 
                         text = 'Trades per stock',
                         bg = 'black',
                         fg = '#e8b923',
                         font = 'Arial 12 bold italic')
        titl2.pack(pady=8,expand=True)
        
        # Iterates through dictionary 2 to print each stocks amount of trades
        for key in dict.keys():
            lab = tk.Label(self,
                           text = key + ': ' + str(dict2[key]),
                           bg = 'black',
                           fg = '#e8b923',
                           font = 'Arial 10 bold italic')
            lab.pack(expand=True)


In [153]:
# Advanced Statistics GUI
class stats_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets GUI Background
        self['bg'] = 'black'
        
        # Uses advanced_statistics method to calculate values
        s,s4,s1,s2 = advanced_statistics()
        
        # Packs total profits into label
        total = tk.Label(self,
                         text = 'Total Profits: $' + str(s),
                         bg = 'black',
                         fg = '#e8b923',
                         font = 'Arial 10 bold italic')
        total.pack(expand=True, pady=6)
        
        # Packs profits per day into label
        perday = tk.Label(self,
                          text = 'Profits per day: $' + str(s1),
                          bg = 'black',
                          fg = '#e8b923',
                          font = 'Arial 10 bold italic')
        perday.pack(expand=True, pady = 6)
        
        
        # Packs profits per trade into label
        pertrade = tk.Label(self,
                            text = 'Profits per trade: $' + str(s4),
                            bg = 'black',
                            fg = '#e8b923',
                            font = 'Arial 10 bold italic')
        pertrade.pack(expand=True, pady=6)
        
        # Packs std of excess returns into label
        std = tk.Label(self, 
                       text = 'Standard Deviation of Returns: ' + str(s2) + '%',
                       bg = 'black',
                       fg = '#e8b923',
                       font = 'Arial 10 bold italic')
        std.pack(expand=True, pady=6)
        
        # Creates frame for Sharpe Ratio
        frame = tk.Frame(self, background = 'black')
        
        # Title for Entry on Bond Yielf
        titl = tk.Label(frame, 
                        text = 'Bond Yield (%)',
                        bg = 'black',
                        fg = '#e8b923',
                        font = 'Arial 10 bold italic')
        titl.grid(row = 0, column = 1)
        
        # Takes entry for bond yield percentage
        self.entry = tk.Entry(frame, 
                              textvariable = 'yield',
                              bg = '#e8b923',
                             borderwidth = 0)
        self.entry.grid(row=1,column = 1)
        
        # Creates Sharpe Ratio button, grids into sharpe frame, calls sharpe_ratio
        button = tk.Button(frame, 
                           text = 'Calculate Sharpe Ratio', 
                           command = self.sharpe_ratio,
                           bg = 'black', 
                           fg = '#e8b923', 
                           font = 'Arial 10 italic',
                           highlightcolor = '#e8b923',
                           highlightbackground = '#e8b923',
                           highlightthickness = 1)
        button.grid(row = 1, column  =0) 
        
        # Packs frame
        frame.pack(expand=True)
        
        self.sharpe_frame = tk.Frame(self,background = 'black')
        self.sharpe_frame.pack(expand=True)
        
    # Method to calculate sharpe ratio
    def sharpe_ratio(self):
    
        # Uses advanced statistics method to gather total capital and stdev 
        total,b,c,stdev = advanced_statistics()
        
        # Calculates Sharpe Ratio
        total = total / sheet['K2'].value
        yieldval = float(self.entry.get())/100
        sharpe = (total - yieldval)/(stdev)
        
        # Creates label for sharpe ratio, packs into self
        lab = tk.Label(self.sharpe_frame, 
                       text = 'Sharpe Ratio: ' + str(sharpe),
                       bg = 'black',
                       fg = '#e8b923',
                       font = 'Arial 10 italic')
        lab.pack(fill=tk.BOTH,expand=True,pady = 8)

In [174]:
class Edit_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets GUI Background color to black
        self['bg'] = 'black'
        
        # Creates 3 frame: Intro, Options, and Cell
        self.intro_frame = tk.Frame(self, background = 'black')
        self.options_frame = tk.Frame(self, background = 'black')
        self.cell_frame = tk.Frame(self, background = 'black')
        
        # Packs First Button into intro frame: Display Sheet for Edits, calls display_excel
        self.display_button = tk.Button(self.intro_frame, 
                                        text = 'Display Excel Sheet for Edits',
                                        command = self.display_excel,
                                        bg = 'black', 
                                        fg = '#e8b923', 
                                        font = 'Arial 10 italic',
                                        highlightcolor = '#e8b923',
                                        highlightbackground = '#e8b923',
                                        highlightthickness = 1)
        self.display_button.pack(padx = 10,pady = 10,expand=True)
        
        # Creates and packs intro label into intro frame
        self.display_label = tk.Label(self.intro_frame, 
                                      text = 'What would you like to edit?',
                                      bg = 'black',
                                      fg = '#e8b923',
                                      font = 'Arial 12 italic')
        self.display_label.pack(padx = 10,pady = 10,expand=True)
        
        # Packs intro frame
        self.intro_frame.pack(pady = 5,expand=True)
        
        # Creates Label for Capital Entry
        self.cap_label = tk.Label(self.options_frame, 
                                  text = 'Enter New Capital',
                                   bg = 'black',
                                   fg = '#e8b923',
                                   font = 'Arial 10 italic')
        self.cap_label.grid(row = 0, column = 1)
        
        # Creates Button for 'Clear Excel' and cooresponding entry spot for new capital
        self.clear_button = tk.Button(self.options_frame,
                                       text = 'Clear Excel',
                                       command = self.clear,
                                       bg = 'black', 
                                       fg = '#e8b923', 
                                       font = 'Arial 10 italic',
                                       highlightcolor = '#e8b923',
                                       highlightbackground = '#e8b923',
                                       highlightthickness = 1)
        self.clear_button.grid(row = 1, column = 0)
        self.clear_entry = tk.Entry(self.options_frame, 
                                    textvariable = 'cap',
                                    bg = '#e8b923',
                                    borderwidth = 0)
        self.clear_entry.grid(row=1,column=1)
        
        # Creates label for remove row
        self.row_label = tk.Label(self.options_frame, 
                                  text = 'Enter Row to Remove',
                                  bg = 'black',
                                  fg = '#e8b923',
                                  font = 'Arial 10 italic')
        self.row_label.grid(row = 2, column = 1)
        
        # Creates, grids button for 'Remove Row' and cooresponding entry spot for row
        self.row_button = tk.Button(self.options_frame,
                                    text = 'Clear Row', 
                                    command = self.clear_row,
                                    bg = 'black', 
                                    fg = '#e8b923', 
                                    font = 'Arial 10 italic',
                                    highlightcolor = '#e8b923',
                                    highlightbackground = '#e8b923',
                                    highlightthickness = 1)
        self.row_button.grid(row = 3, column = 0)
        self.row_entry = tk.Entry(self.options_frame,
                                  textvariable = 'row',
                                  bg = '#e8b923',
                                  borderwidth =0)
        self.row_entry.grid(row=3,column=1)
        
        # Packs options frame
        self.options_frame.pack(expand=True)
        
        # Creates label for cell and value entry
        self.enter_cell = tk.Label(self.cell_frame, 
                                   text = 'Enter Cell',
                                   bg = 'black',
                                   fg = '#e8b923',
                                   font = 'Arial 10 italic')
        self.enter_cell.grid(row = 0, column = 1)
        self.enter_val = tk.Label(self.cell_frame,
                                  text = 'Enter New Value',
                                   bg = 'black',
                                   fg = '#e8b923',
                                   font = 'Arial 10 italic')
        self.enter_val.grid(row = 0, column =2)
        
        # Creates, grids button for alter cell and entry spots for cell and value
        self.cell_button = tk.Button(self.cell_frame, 
                                     text = 'Alter Specific Cell', 
                                     command = self.alter,
                                     bg = 'black', 
                                     fg = '#e8b923', 
                                     font = 'Arial 10 italic',
                                     highlightcolor = '#e8b923',
                                     highlightbackground = '#e8b923',
                                     highlightthickness = 1)
        self.cell_button.grid(row = 1, column = 0)
        self.cell = tk.Entry(self.cell_frame, 
                             textvariable = 'Cell',
                             bg = '#e8b923', 
                             borderwidth =0,
                             highlightcolor = 'black',
                             highlightbackground = 'black',
                             highlightthickness = 1)
        self.cell.grid(row = 1, column = 1)
        self.val = tk.Entry(self.cell_frame,
                            textvariable = 'Value',
                            bg = '#e8b923', 
                            borderwidth = 0,
                            highlightcolor = 'black',
                            highlightbackground = 'black',
                            highlightthickness = 1)
        self.val.grid(row = 1, column = 2)
        
        # Packs cell frame
        self.cell_frame.pack(expand=True)
        
        # Creates and packs confirmation frame
        self.conf = tk.Frame(self, background = 'black')
        self.conf.pack()
        
    # Calls display_excel GUI
    def display_excel(self):
        ex_gui = Excel_GUI()
        ex_gui.mainloop()
    
    # Clear Excel method
    def clear(self):
        
        # Deletes the entire sheet
        for row in range(0,sheet.max_row+100):
            sheet.delete_rows(row)
        
        # Adds new column headers, gets the capital entry values, place that into sheet
        sheet['K2'] = float(self.clear_entry.get())
        sheet['A1'] = 'Ticker'
        sheet['B1'] = 'Date Bought'
        sheet['C1'] = 'Quantity Purchased'
        sheet["D1"] = 'Fulfillment Price'
        sheet['E1'] = 'Sell Date'
        sheet['F1'] = 'Quantity Sold'
        sheet['G1'] = 'Fulfillment Price'
        sheet['H1'] = 'Quantity Remaining'
        sheet["I1"] = 'P/L Percentage'
        sheet['J1'] = 'P/L Amount'
        sheet['K1'] = 'Capital'
        sheet['A2'] = 'Starting Balance'
        
        # Creates and packs confirmation label into frame
        lab = tk.Label(self.conf,
                       text = 'Excel Sheet Cleared',
                       bg = 'black',
                       fg = '#e8b923',
                       font = 'Arial 10 italic')
        lab.pack(expand=True,pady=4)
        
        # Packs frame
        clear_frame.pack(expand=True)
    
    # Clear row method
    def clear_row(self):
        
        # Captures entered row
        row = int(self.row_entry.get())
        
        # Deletes the Row
        sheet.delete_rows(row)
        
        # Prints Confirmation
        lab2 = tk.Label(self.conf,
                        text = 'Row ' + str(row) + ' has been cleared!',
                        bg = 'black',
                        fg = '#e8b923',
                        font = 'Arial 10 italic')
        lab2.pack(expand=True,pady=4)
    
    # Alter Specific Cell Method
    def alter(self):
        # Finds the cell it asked for
        cell = str(self.cell.get())
        col = cell[0]
        
        # If the cell is a date, convert the value to datetime
        if(col == 'B' or col == 'E'):
            unval = str(self.val.get())
            val = format_date(unval)
            sheet[cell] = val
            
        # If the cell is a string, convert the entered ticker to a string
        elif(col == 'A'):
            val = str(self.val.get())
            sheet[cell] = val
        
        # If the cell is a integer, convert the entry to an integer
        elif(col == 'C' or col == 'F' or col =='H'):
            val = int(self.val.get())
            sheet[cell] = val
            
        # If the cell is a float, convert entered date to a float
        else:
            val = float(self.val.get())
            sheet[cell] = val
        
        # Creates, packs confirmation label
        lab3 = tk.Label(self.conf,
                        text = 'Cell' + str(cell) + ' has been changed!',
                        bg = 'black',
                        fg = '#e8b923',
                        font = 'Arial 10 italic')
        lab3.pack(expand=True,pady=4)

In [175]:
# Excel GUI
class Excel_GUI(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        
        # Sets background
        self['bg'] = 'black'
        
        # Creates List of Column Headers
        alph = ['A','B','C','D','E','F','G','H','I','J','K']
        self.sheet_frame = tk.Frame(self, background = 'black')
        
        # Packs Column Headers
        for col in range(0,len(alph)):
            fram = tk.Frame(self.sheet_frame,
                           background = 'black')
            lab = tk.Label(fram, 
                           text = alph[col],
                           bg = 'black',
                           fg = '#e8b923',
                           borderwidth = 1)
            lab.pack()
            fram.grid(row=0,column=col+1)
        
        # Iterates through each row for the sheet
        for row in range(1,sheet.max_row+1):
            fram = tk.Frame(self.sheet_frame,
                           background = 'black')
            # Adds row labels
            box = tk.Label(fram, 
                           text = str(row),
                           bg = 'black',
                           fg = '#e8b923',
                           borderwidth = 1)
            box.pack()
            fram.grid(row =row, column = 0)
            
            # Iterates through each row
            for col in range(1,13):
                
                fram = tk.Frame(self.sheet_frame,
                           background = 'black')
                
                # Assigns cells the values
                cell = tk.Label(fram,
                                text =  sheet.cell(row = row, column = col).value,
                                bg = 'black',
                                fg = '#e8b923',
                                borderwidth = 0)
                cell.pack()
                fram.grid(row=row,column = col)
                
        # Packs everything
        self.sheet_frame.pack(side = tk.TOP, fill = tk.BOTH, expand=True)

# Runs the GUI

After GUI is completed running, date is automatically exported at 'Finance.xlsx'

In [176]:
# Runs the GUI!
root = tk.Tk()
main_gui = Main_GUI(root)
root.mainloop()

# Saves as excel sheet
finance.save('Finance.xlsx')